# Small

In [1]:
import simuOpt
simuOpt.setOptions(alleleType='short', numThreads=4, quiet=True)
import simuPOP as sim
import pandas as pd
from saegus import breed, operators, simulate, analyze, parse, parameters
import shelve
import numpy as np
import random
import h5py
import collections as col
np.set_printoptions(suppress=True, precision=3)
pd.options.display.float_format = '{:.8f}'.format

In [2]:
small = analyze.Study('small')

In [3]:
run_id = 'small'
generations_of_random_mating = 10
number_of_qtl = 20
number_of_replicates = 10
founders = [[2, 26], [3, 25], [4, 24], [5, 23]]
os_per_pair = 500
recombination_rates = [0.01]*1478

In [4]:
prefounders = sim.loadPopulation('bia_prefounders.pop')

In [5]:
prefounders.infoFields()

('ind_id',
 'father_id',
 'mother_id',
 'fitness',
 'p',
 'g',
 'generation',
 'replicate')

In [6]:
sim.tagID(prefounders, reset=True)

In [7]:
prefounders.popSize()

26

In [8]:
multi_prefounders = sim.Simulator(prefounders, 10, stealPops=False)

In [9]:
magic = breed.MAGIC(multi_prefounders, founders, recombination_rates)

In [10]:
magic.generate_f_one(founders, os_per_pair)

In [11]:
mrc = breed.MultiRandomCross(multi_prefounders, 4, 500)

In [12]:
mother_choices, father_choices = mrc.determine_random_cross()

In [13]:
multi_snd_ord_chooser = breed.MultiSecondOrderPairIDChooser(
    mother_choices, father_choices)

In [14]:
multi_prefounders.evolve(
    matingScheme=sim.HomoMating(
        sim.PyParentsChooser(multi_snd_ord_chooser.snd_ord_id_pairs),
        sim.OffspringGenerator(ops=[
            sim.IdTagger(),
            sim.PedigreeTagger(),
            sim.Recombinator(rates=0.01)
        ],
            numOffspring=1),
        subPopSize=[2000],
    ),
    gen=1,
)

(1, 1, 1, 1, 1, 1, 1, 1, 1, 1)

In [15]:
final_mrc = breed.MultiRandomCross(multi_prefounders, 2, 1000)

In [16]:
final_mothers, final_fathers = final_mrc.determine_random_cross()

In [17]:
final_multi_snd_ord_chooser = breed.MultiSecondOrderPairIDChooser(
    final_mothers, final_fathers)

In [18]:
multi_prefounders.evolve(
    matingScheme=sim.HomoMating(
        sim.PyParentsChooser(final_multi_snd_ord_chooser.snd_ord_id_pairs),
        sim.OffspringGenerator(ops=[
            sim.IdTagger(),
            sim.PedigreeTagger(),
            sim.Recombinator(rates=0.01)
        ],
            numOffspring=1),
        subPopSize=[2000],
    ),
    gen=1,
)

(1, 1, 1, 1, 1, 1, 1, 1, 1, 1)

# Random Mating Phase

In [19]:
multi_prefounders.evolve(
    matingScheme=sim.RandomMating(ops=[
            sim.IdTagger(),
            sim.PedigreeTagger(),
            sim.Recombinator(rates=0.01)
        ],
        subPopSize=[2000]),
    gen=10,
)

(10, 10, 10, 10, 10, 10, 10, 10, 10, 10)

In [20]:
sample_size = 500

In [21]:
sample_library = small.collect_samples(multi_prefounders, [sample_size])

In [22]:
for rep_id, sample_list in sample_library.items():
    sim.stat(sample_list[0], numOfSegSites=sim.ALL_AVAIL, vars=['numOfSegSites', 'segSites'])
    sim.stat(sample_list[0], alleleFreq=sim.ALL_AVAIL)

In [23]:
sample = sample_library[0][0]

In [24]:
astates = small.gather_allele_data(sample)

In [25]:
alleles = np.array([astates[:, 1], astates[:, 2]]).T

In [26]:
segregating_loci = np.array(sample.dvars().segSites)

In [27]:
trait = parameters.Trait()

In [28]:
qtl = sorted(list(random.sample(list(segregating_loci), 20)))

In [29]:
allele_effects = trait.construct_allele_effects_table(alleles, qtl, random.expovariate, 1)

In [30]:
ae_array = trait.construct_ae_array(allele_effects, qtl)

# Storing Data

In [31]:
small_data = h5py.File('small_data.hdf5')

In [55]:
small_data['allele/states'] = astates
small_data['segregating_loci'] = segregating_loci
small_data['qtl'] = np.array(qtl)
small_data['allele/effects'] = allele_effects
#small_data['recombination_rates'] = np.array(recombination_rates)

RuntimeError: Unable to create link (Name already exists)

In [54]:
recombination_rates

[0.01,
 0.01,
 0.01,
 0.01,
 0.01,
 0.01,
 0.01,
 0.01,
 0.01,
 0.01,
 0.01,
 0.01,
 0.01,
 0.01,
 0.01,
 0.01,
 0.01,
 0.01,
 0.01,
 0.01,
 0.01,
 0.01,
 0.01,
 0.01,
 0.01,
 0.01,
 0.01,
 0.01,
 0.01,
 0.01,
 0.01,
 0.01,
 0.01,
 0.01,
 0.01,
 0.01,
 0.01,
 0.01,
 0.01,
 0.01,
 0.01,
 0.01,
 0.01,
 0.01,
 0.01,
 0.01,
 0.01,
 0.01,
 0.01,
 0.01,
 0.01,
 0.01,
 0.01,
 0.01,
 0.01,
 0.01,
 0.01,
 0.01,
 0.01,
 0.01,
 0.01,
 0.01,
 0.01,
 0.01,
 0.01,
 0.01,
 0.01,
 0.01,
 0.01,
 0.01,
 0.01,
 0.01,
 0.01,
 0.01,
 0.01,
 0.01,
 0.01,
 0.01,
 0.01,
 0.01,
 0.01,
 0.01,
 0.01,
 0.01,
 0.01,
 0.01,
 0.01,
 0.01,
 0.01,
 0.01,
 0.01,
 0.01,
 0.01,
 0.01,
 0.01,
 0.01,
 0.01,
 0.01,
 0.01,
 0.01,
 0.01,
 0.01,
 0.01,
 0.01,
 0.01,
 0.01,
 0.01,
 0.01,
 0.01,
 0.01,
 0.01,
 0.01,
 0.01,
 0.01,
 0.01,
 0.01,
 0.01,
 0.01,
 0.01,
 0.01,
 0.01,
 0.01,
 0.01,
 0.01,
 0.01,
 0.01,
 0.01,
 0.01,
 0.01,
 0.01,
 0.01,
 0.01,
 0.01,
 0.01,
 0.01,
 0.01,
 0.01,
 0.01,
 0.01,
 0.01,
 0.01,
 0.01,
 0.01,

In [33]:
for rep, sample_list in sample_library.items():
    small_data['allele/frequency/replicate/' + str(rep)] = small.gather_allele_frequencies(sample_list[0], astates)
    operators.calculate_g(sample_list[0], ae_array)
    operators.calculate_error_variance(sample_list[0], 0.7)
    operators.calculate_p(sample_list[0])
    small_data['trait/g/replicate/' + str(rep)] = np.array([sample_list[0].indInfo('ind_id'), 
                                                            sample_list[0].indInfo('g')]).T
    small_data['trait/p/replicate/' + str(rep)] = np.array([sample_list[0].indInfo('ind_id'),
                                                          sample_list[0].indInfo('p')]).T
    

In [34]:
small_data['trait'].attrs['heritability'] = np.array([0.7])

# Correcting the Config File

In [35]:
minor_alleles = np.array(small_data['allele/states'])[:, 3]

In [36]:
name = 'small_0'

In [37]:
from os import path

In [38]:
indir = '/home/vakanas/tassel-5-standalone/input'
outdir = '/home/vakanas/tassel-5-standalone/output'

In [39]:
for rep, sample_list in sample_library.items():
    name = small.run_id + '_' + str(rep)
    minor_allele_fs = np.array(small_data['allele/frequency/replicate/' + str(rep)])[segregating_loci, 3]
    gwas = analyze.GWAS(sample_list[0], segregating_loci, minor_alleles, 'small')
    cm = gwas.calculate_count_matrix(count_matrix_file_name=path.join(indir, name+'_count_matrix.txt'))
    ps, svd = gwas.pop_struct_eigendecomp(cm)
    gwas.population_structure_formatter(ps, svd, number_of_pcs=2, 
                                        pop_struct_file_name=path.join(indir, name+'_structure_matrix.txt'))
    gwas.trait_formatter(trait_file_name=path.join(indir, name+'_phenotype_vector.txt'))
    gwas.calc_kinship_matrix(cm, minor_allele_fs, kinship_matrix_file_name=path.join(indir, name+'_kinship_matrix.txt'))
    gwas.hapmap_formatter(hapmap_file_name=path.join(indir, name+'_simulated_hapmap.txt'))
    gwas.single_gen_multi_rep_tassel_config(rep, 'gwas_pipeline.xml', output_prefix=name+'_out_')

# Run TASSEL at This Point

Contents of bash script to automate TASSEL via configuration files.

### simulated_mlm.sh

```bash
#!/bin/bash


echo "Run ID: $1, Number of Replicates $2"
run_id=$1
number_of_replicates=$2
final_rep_index="$((number_of_replicates - 1))"

echo "Beginning TASSEL analysis of Run ID: $run_id"
echo "Number of Replicates: $number_of_replicates"
echo "First configuration file: small_0_gwas_pipeline.xml"onca
echo "Final configuration file: small_"$final_rep_index"_gwas_pipeline.xml"

for i in `seq 0 $final_rep_index`
do
    config_file_name=$run_id$i"_gwas_pipeline.xml"
    echo "$config_file_name"
    ./run_pipeline.pl -Xmx6g -configFile $config_file_name
done


```

### Example output: 
+ small_0_out_1.txt
+ small_0_out_2.txt
+ small_0_out_3.txt
+ small_1_out_1.txt
+ ...
+ small_9_out_3.txt

# Use R Qvalue package to get Qvalues

Contents of R script to obtain qvalues for p column of TASSEL results

```R
#!/usr/bin/env Rscript

library(qvalue)
library(ggplot2)
library(gap)

args = commandArgs(trailingOnly=TRUE)

# test to determine if the file name parameter is supplied to the script
if (length(args)==0) {
  stop("At least one argument must be suppled (input file).\n", call.=FALSE)
}
#setwd("/home/vakanas/tassel-5-standalone/output")  

run_id = args[1]
file_name_match_pattern = paste(run_id, "(.*)_2.txt", sep='')
file_names = list.files(pattern = file_name_match_pattern)

for(n in file_names) {
    print(n)
    input_file_name = n
    run_id_prefix_terminus = nchar(input_file_name) - 5
    run_id_prefix = substring(input_file_name, 1, run_id_prefix_terminus)
    output_file_name = paste(run_id_prefix, 'q_values.txt', sep='')
    print(output_file_name)
    results_header = scan(input_file_name, what="character", nlines=1, sep="\t")
    gwas_results = read.table(input_file_name, header=F, row.names = NULL, skip=2)
    colnames(gwas_results) = results_header
    pvalues = gwas_results$p
    qobj = qvalue(p = pvalues)
    qvalues = data.frame(qobj$qvalues)
    colnames(qvalues) = "q"
    rownames(qvalues) = gwas_results$Marker
    write.table(qvalues, output_file_name, sep="\t")
}

```

# Analysis of TASSEL Results: Comutation of Power & FPR

## Subsetting Raw TASSEL Results and Data Storage
    Each replicate has an associated set of TASSEL output files. The raw
    results are modified and stored in the run's HDF5 file

## Statistical Power and False Positive Rate

In [213]:
power_and_fprs = np.zeros((10, 3))
for i in range(10):
    tassel_results = pd.read_csv('/home/vakanas/tassel-5-standalone/small_'+str(i)+'_out_2.txt', 
                             sep='\t', 
                             skiprows=[1], 
                             index_col='Marker')
    tassel_results.drop(labels=['Trait', 'Pos', 'dom_effect', 
                            'dom_F', 'dom_p', 'Genetic Var',
                            'Residual Var', '-2LnLikelihood', 
                            'add_effect', 'add_F', 'add_p', 'errordf', 'MarkerR2'], 
                        axis=1, inplace=True)
    qvalues = pd.read_csv('/home/vakanas/tassel-5-standalone/small_'+str(i)+'_out_q_values.txt', 
                          sep='\t', index_col=0)
    truncated_plus_q = tassel_results.join(qvalues)
    small_data['tassel/test/replicate/'+str(i)] = np.array(truncated_plus_q)
    power = len(qvalues.ix[qvalues['q'] < 0.05])/len(qtl)
    false_positive_rate = sum(allele_effects[np.array(qvalues.ix[qvalues['q'] < 0.05].index)][1] == 0)
    power_and_fprs[i, 0] = i
    power_and_fprs[i, 1] = power
    power_and_fprs[i, 2] = false_positive_rate

In [222]:
power_and_fprs[:, 1].mean()

0.215

In [223]:
power_and_fprs[:, 2].mean()

0.59999999999999998

In [224]:
np.var(power_and_fprs[:, 1])

0.0015249999999999999

## Allele Effects: Estimated vs. Actual
    Purpose of this table to compare TASSEL's effect size estimates
    with the known true values we assigned.

In [383]:
raw_effect_estimates = pd.read_csv('/home/vakanas/tassel-5-standalone/small_0_out_3.txt', 
                                   sep='\t')

In [362]:
#raw_effect_estimates.drop(labels=['Trait', 'Locus', 'Site', 'Obs'], axis=1, inplace=True)

In [395]:
grouped = raw_effect_estimates.groupby('Marker')

In [412]:
mapping_of_frames = dict(list(grouped))

In [455]:
for allele in mapping_of_frames[13].Allele:
    print(allele,
         iupac_genotype_codes[allele],
         snp_to_int[iupac_genotype_codes[allele][0]],
         snp_to_int[iupac_genotype_codes[allele][1]],
         float(mapping_of_frames[13].ix[mapping_of_frames[13].ix[:, 'Allele'] == allele].Effect),
         ae_array[13, snp_to_int[iupac_genotype_codes[allele][0]]] +
         ae_array[13, snp_to_int[iupac_genotype_codes[allele][1]]]
         )

A AA 0 0 0.48413999999999996 0.18744407135
G GG 2 2 0.9191600000000001 0.7631182083
R AG 0 2 0.0 0.475281139825


In [534]:
table_columns = [
    'Marker',
    'G1',
    'P(G1)',
    'e[G1]*',
    'e[G1]',
    '|e[G1]* - e[G1]|',
    'G2',
    'P(G2)',
    'e[G2]*',
    'e[G2]',
    '|e[G2]* - e[G2]|',
    'G3',
    'P(G3)',
    'e[G3]*',
    'e[G3]',
    '|e[G3]* - e[G3]|'
]

In [535]:
genotypic_effects = pd.DataFrame(np.zeros((943, len(table_columns))), index=segregating_loci, columns=table_columns)

In [538]:
genotypic_effects.Marker = segregating_loci

In [536]:
for locus in qtl:
    for idx, allele in enumerate(mapping_of_frames[locus].Allele):
        current_genotype = 'G'+str(idx+1)
        estimated_genotypic_effect_key = 'e[G'+str(idx+1)+']*'
        estimated_effect =  float(mapping_of_frames[locus].ix[mapping_of_frames[locus].ix[:, 'Allele'] == allele].Effect)
        true_genotypic_effect_key = 'e[G'+str(idx+1)+']'
        true_genotypic_effect = ae_array[locus, snp_to_int[iupac_genotype_codes[allele][0]]] + ae_array[locus, snp_to_int[iupac_genotype_codes[allele][1]]]
        genotypic_effects.ix[locus, current_genotype] = iupac_genotype_codes[allele]
        genotypic_effects.ix[locus, estimated_genotypic_effect_key] = estimated_effect
        genotypic_effects.ix[locus, true_genotypic_effect_key] = true_genotypic_effect
        frq_genotype_key = 'P(G'+str(idx+1)+')'
        frq_genotype = sample.dvars().genoFreq[locus][snp_to_int[iupac_genotype_codes[allele][0]], 
                                                      snp_to_int[iupac_genotype_codes[allele][1]]]
        genotypic_effects.ix[locus, frq_genotype_key] = frq_genotype
        '|e[G1]* - e[G1]|'
        abs_difference_key = '|e['+current_genotype+']* - e['+current_genotype+']|'
        abs_difference = abs(estimated_effect - true_genotypic_effect)
        genotypic_effects.ix[locus, abs_difference_key] = abs_difference

In [539]:
genotypic_effects.ix[qtl]

,Marker,G1,P(G1),e[G1]*,e[G1],|e[G1]* - e[G1]|,G2,P(G2),e[G2]*,e[G2],|e[G2]* - e[G2]|,G3,P(G3),e[G3]*,e[G3],|e[G3]* - e[G3]|
13,13,AA,0.01000000,0.48414000,0.18744407,0.29669593,GG,0.80800000,0.91916000,0.76311821,0.15604179,AG,0.08200000,0.00000000,0.47528114,0.47528114
27,27,AA,0.85000000,-0.58152000,0.05457335,0.63609335,CC,0.00800000,-0.19921000,0.17904314,0.37825314,AC,0.06200000,0.00000000,0.11680825,0.11680825
198,198,CC,0.79400000,0.65672000,2.30235895,1.64563895,TT,0.01400000,-2.71790000,1.91281887,4.63071887,CT,0.09400000,0.00000000,2.10758891,2.10758891
278,278,AA,0.89600000,0.79567000,0.76805141,0.02761859,TT,0.00400000,-4.15710000,0.11820734,4.27530734,AT,0.04800000,0.00000000,0.44312937,0.44312937
363,363,CC,0.11200000,-2.82820000,0.93035723,3.75855723,GG,0.41800000,0.69624000,4.21261343,3.51637343,CG,0.25400000,0.00000000,2.57148533,2.57148533
436,436,AA,0.04600000,0.62447000,1.89044825,1.26597825,GG,0.57800000,1.34126000,2.21149876,0.87023876,AG,0.17600000,0.00000000,2.05097350,2.05097350
441,441,CC,0.72800000,-0.09090100,1.23518170,1.32608270,GG,0.01400000,-1.17180000,1.28070454,2.45250454,CG,0.12600000,0.00000000,1.25794312,1.25794312
550,550,CC,0.26600000,0.53248000,2.33540875,1.80292875,TT,0.26000000,-0.31965000,1.93832716,2.25797716,CT,0.24200000,0.00000000,2.13686795,2.13686795
604,604,AA,0.06800000,-1.95320000,0.70997998,2.66317998,TT,0.58200000,3.10303000,5.63435513,2.53132513,AT,0.16400000,0.00000000,3.17216756,3.17216756
735,735,CC,0.62200000,-2.15070000,0.97583248,3.12653248,TT,0.04200000,3.24422000,6.23761538,2.99339538,CT,0.18200000,0.00000000,3.60672393,3.60672393


In [542]:
genotypic_effects.ix[13]

Marker                     13
G1                         AA
P(G1)              0.01000000
e[G1]*             0.48414000
e[G1]              0.18744407
|e[G1]* - e[G1]|   0.29669593
G2                         GG
P(G2)              0.80800000
e[G2]*             0.91916000
e[G2]              0.76311821
|e[G2]* - e[G2]|   0.15604179
G3                         AG
P(G3)              0.08200000
e[G3]*             0.00000000
e[G3]              0.47528114
|e[G3]* - e[G3]|   0.47528114
Name: 13, dtype: object

In [546]:
genotypic_effects.cov()

,Marker,P(G1),e[G1]*,e[G1],|e[G1]* - e[G1]|,P(G2),e[G2]*,e[G2],|e[G2]* - e[G2]|,P(G3),e[G3]*,e[G3],|e[G3]* - e[G3]|
Marker,179175.47761470,-1.08114613,1.20835222,2.54182687,3.23410884,1.51414456,5.91821356,1.96087844,-3.00192257,0.11288886,0.00000000,2.25135265,2.25135265
P(G1),-1.08114613,0.00449625,0.00048002,0.00893229,0.00886970,0.00041444,-0.00373704,0.01060402,0.01517941,0.00091092,0.00000000,0.00976816,0.00976816
e[G1]*,1.20835222,0.00048002,0.06560147,0.02578700,-0.01409196,0.00049333,-0.03509374,-0.03540233,-0.00078978,-0.00056369,0.00000000,-0.00480767,-0.00480767
e[G1],2.54182687,0.00893229,0.02578700,0.06381292,0.05057214,0.00908873,-0.00583537,0.04953946,0.05583659,0.00377384,0.00000000,0.05667619,0.05667619
|e[G1]* - e[G1]|,3.23410884,0.00886970,-0.01409196,0.05057214,0.07732169,0.01471800,0.02993258,0.09607732,0.06793354,0.00529999,0.00000000,0.07332473,0.07332473
P(G2),1.51414456,0.00041444,0.00049333,0.00908873,0.01471800,0.00602487,0.00804611,0.01945778,0.01251449,0.00103354,0.00000000,0.01427325,0.01427325
e[G2]*,5.91821356,-0.00373704,-0.03509374,-0.00583537,0.02993258,0.00804611,0.07227777,0.05958193,-0.00924774,0.00209489,0.00000000,0.02687328,0.02687328
e[G2],1.96087844,0.01060402,-0.03540233,0.04953946,0.09607732,0.01945778,0.05958193,0.15636556,0.09868805,0.00695890,0.00000000,0.10295251,0.10295251
|e[G2]* - e[G2]|,-3.00192257,0.01517941,-0.00078978,0.05583659,0.06793354,0.01251449,-0.00924774,0.09868805,0.10927539,0.00526563,0.00000000,0.07726232,0.07726232
P(G3),0.11288886,0.00091092,-0.00056369,0.00377384,0.00529999,0.00103354,0.00209489,0.00695890,0.00526563,0.00047734,0.00000000,0.00536637,0.00536637


In [505]:
sim.stat(sample, genoFreq=list(segregating_loci))

In [506]:
sample.dvars().genoFreq[1198]

defdict({(1, 1): 0.004, (1, 3): 0.062, (3, 1): 0.052, (3, 3): 0.882})

In [450]:
mapping_of_frames[13].ix[mapping_of_frames[13].ix[:, 'Allele'] == 'A']

,Trait,Marker,Locus,Site,Allele,Effect,Obs
26,sim,13,1,9,A,0.48414000,5


In [385]:
first_genotype = raw_effect_estimates.ix[::3]

In [394]:
936+935+935

2806

In [389]:
first_genotype.shape

(936, 7)

In [386]:
second_genotype = raw_effect_estimates.ix[1::3]

In [388]:
second_genotype.shape

(935, 7)

In [390]:
third_genotype = raw_effect_estimates.ix[2::3]

In [392]:
third_genotype.shape

(935, 7)

In [ ]:
flattened_effect_table

In [314]:
first_homozygo = np.array(raw_effect_estimates)[0::3, 1:]

In [324]:
frame_first_homozygo = pd.DataFrame(first_homozygo, index=first_homozygo[:, 0])

In [328]:
frame_first_homozygo.ix[13, 3]

'G'

In [333]:
ae_array[13, snp_to_int[iupac_genotype_codes[frame_first_homozygo.ix[13, 3]][0]]]

0.381559104149788

In [349]:
true_genotypic_effects = np.zeros((1478, 6, 6))

In [350]:
true_genotypic_effects[13, 2, 2] = ae_array[13, snp_to_int[iupac_genotype_codes[frame_first_homozygo.ix[13, 3]][0]]]*2

In [351]:
true_genotypic_effects[13]

array([[ 0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ],
       [ 0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ],
       [ 0.   ,  0.   ,  0.763,  0.   ,  0.   ,  0.   ],
       [ 0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ],
       [ 0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ],
       [ 0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ]])

In [318]:
second_homozygo = np.array(raw_effect_estimates)[1::3, 1:]

In [343]:
frame_second_homozygo = pd.DataFrame(second_homozygo, index=second_homozygo[:, 0])

In [347]:
true_genotypic_effects = ae_array[13, snp_to_int[iupac_genotype_codes[frame_second_homozygo.ix[13, 3]][0]]]*2

In [357]:
iupac_genotype_codes[frame_second_homozygo.ix[13, 3]]

'AG'

In [353]:
snp_to_int[iupac_genotype_codes[frame_second_homozygo.ix[13, 3]]]

KeyError: 'AG'

In [348]:
true_genotypic_effects[13, ]

0.1874440713495259

In [319]:
second_homozygo

array([[1, 1, 0, 'T', -0.46931999999999996, 16],
       [2, 1, 1, 'T', -0.4142, 375],
       [3, 1, 2, 'G', -3.0823, 8],
       ..., 
       [1474, 10, 940, 'A', 0.81284, 298],
       [1475, 10, 941, 'A', -0.48391999999999996, 32],
       [1476, 10, 942, 'A', -3.5997, 12]], dtype=object)

In [317]:
heterozygo = np.array(raw_effect_estimates)[2::3, 1:]

In [320]:
heterozygo

array([[1, 1, 0, 'Y', 0.0, 152],
       [2, 1, 1, 'Y', 0.0, 117],
       [3, 1, 2, 'R', 0.0, 124],
       ..., 
       [1474, 10, 940, 'G', -2.0238, 29],
       [1475, 10, 941, 'C', -0.07948999999999999, 258],
       [1476, 10, 942, 'T', -0.022318, 379]], dtype=object)

In [255]:
heterozygo

array([['sim', 1, 1, ..., 'Y', 0.0, 152],
       ['sim', 2, 1, ..., 'Y', 0.0, 117],
       ['sim', 3, 1, ..., 'R', 0.0, 124],
       ..., 
       ['sim', 1474, 10, ..., 'G', -2.0238, 29],
       ['sim', 1475, 10, ..., 'C', -0.07948999999999999, 258],
       ['sim', 1476, 10, ..., 'T', -0.022318, 379]], dtype=object)

In [257]:
iupac_genotype_codes = {
    'A': 'AA',
    0: 'A',
    'AA': 'A',
    1: 'C',
    'C': 'CC',
    'CC': 'C',
    2: 'G',
    'G': 'GG',
    'GG': 'G',
    3: 'T',
    'T': 'TT',
    'TT': 'T',
    'R': 'AG',
    'AG': 'R',
    'Y': 'CT',
    'CT': 'Y',
    'S': 'CG',
    'CG': 'S',
    'W': 'AT',
    'AT': 'W',
    'K': 'GT',
    'GT': 'K',
    'M': 'AC',
    'AC': 'M',
    5: '+',
    '+': '++',
    '++': '+',
    '0': '+-',
    '+-': '0',
    4: '-',
    '-': '--',
    '--': '-',
}

In [267]:
snp_to_int = {
    'A': 0,
    'C': 1,
    'G': 2,
    'T': 3,
    '-': 4,
    '+': 5
}

In [265]:
estimated_effect_sizes = np.zeros((943, 5))

In [271]:
alphabetical_genotype = iupac_genotype_codes[first_homozygo[0, 4]]

In [273]:
snp_to_int[alphabetical_genotype[0]]

1

In [279]:
qtl

[13,
 27,
 198,
 278,
 363,
 436,
 441,
 550,
 604,
 735,
 853,
 1035,
 1041,
 1189,
 1193,
 1197,
 1198,
 1312,
 1330,
 1432]

In [282]:
raw_effect_estimates

,Trait,Marker,Locus,Site,Allele,Effect,Obs
0,sim,1,1,0,C,-0.04411200,332
1,sim,1,1,0,T,-0.46932000,16
2,sim,1,1,0,Y,0.00000000,152
3,sim,2,1,1,C,-0.31837000,8
4,sim,2,1,1,T,-0.41420000,375
5,sim,2,1,1,Y,0.00000000,117
6,sim,3,1,2,A,-0.19867000,368
7,sim,3,1,2,G,-3.08230000,8
8,sim,3,1,2,R,0.00000000,124
9,sim,4,1,3,G,-0.26613000,456


In [294]:
iupac_genotype_codes[iupac_genotype_codes[first_homozygo[13, 4]]]

'T'

In [307]:
iupac_genotype_codes[first_homozygo[9, 4]]

'GG'

In [308]:
snp_to_int['G']

2

In [311]:
ae_array[13, 2]*2

0.763118208299576

In [295]:
iupac_genotype_codes['T']

'TT'

In [296]:
snp_to_int['T']

3

In [289]:
allele_effects[13]

array([ 13.   ,   0.   ,   0.094,   2.   ,   0.382])

In [292]:
ae_array[26, ]

array([ 0.,  0.,  0.,  0.,  0.,  0.])

In [293]:
ae_array[qtl]

array([[ 0.094,  0.   ,  0.382,  0.   ,  0.   ,  0.   ],
       [ 0.027,  0.09 ,  0.   ,  0.   ,  0.   ,  0.   ],
       [ 0.   ,  1.151,  0.   ,  0.956,  0.   ,  0.   ],
       [ 0.384,  0.   ,  0.   ,  0.059,  0.   ,  0.   ],
       [ 0.   ,  0.465,  2.106,  0.   ,  0.   ,  0.   ],
       [ 0.945,  0.   ,  1.106,  0.   ,  0.   ,  0.   ],
       [ 0.   ,  0.618,  0.64 ,  0.   ,  0.   ,  0.   ],
       [ 0.   ,  1.168,  0.   ,  0.969,  0.   ,  0.   ],
       [ 0.355,  0.   ,  0.   ,  2.817,  0.   ,  0.   ],
       [ 0.   ,  0.488,  0.   ,  3.119,  0.   ,  0.   ],
       [ 0.046,  0.   ,  0.811,  0.   ,  0.   ,  0.   ],
       [ 0.   ,  0.34 ,  0.   ,  0.826,  0.   ,  0.   ],
       [ 1.356,  0.   ,  0.   ,  0.674,  0.   ,  0.   ],
       [ 0.164,  0.   ,  2.222,  0.   ,  0.   ,  0.   ],
       [ 0.   ,  0.166,  0.037,  0.   ,  0.   ,  0.   ],
       [ 0.904,  0.739,  0.   ,  0.   ,  0.   ,  0.   ],
       [ 0.   ,  0.847,  0.   ,  1.763,  0.   ,  0.   ],
       [ 0.002,  0.   ,  0.838,

In [312]:
even_effects

array([['sim', 1, 1, ..., 'C', -0.044112, 332],
       ['sim', 2, 1, ..., 'C', -0.31837, 8],
       ['sim', 3, 1, ..., 'A', -0.19867, 368],
       ..., 
       ['sim', 1474, 10, ..., 'R', 0.0, 173],
       ['sim', 1475, 10, ..., 'M', 0.0, 210],
       ['sim', 1476, 10, ..., 'W', 0.0, 109]], dtype=object)

In [246]:
odd_effects.shape

(935, 7)

In [103]:
allele_effects[qtl]

array([[   13.   ,     0.   ,     0.094,     2.   ,     0.382],
       [   27.   ,     0.   ,     0.027,     1.   ,     0.09 ],
       [  198.   ,     1.   ,     1.151,     3.   ,     0.956],
       [  278.   ,     0.   ,     0.384,     3.   ,     0.059],
       [  363.   ,     1.   ,     0.465,     2.   ,     2.106],
       [  436.   ,     0.   ,     0.945,     2.   ,     1.106],
       [  441.   ,     1.   ,     0.618,     2.   ,     0.64 ],
       [  550.   ,     1.   ,     1.168,     3.   ,     0.969],
       [  604.   ,     0.   ,     0.355,     3.   ,     2.817],
       [  735.   ,     1.   ,     0.488,     3.   ,     3.119],
       [  853.   ,     0.   ,     0.046,     2.   ,     0.811],
       [ 1035.   ,     1.   ,     0.34 ,     3.   ,     0.826],
       [ 1041.   ,     0.   ,     1.356,     3.   ,     0.674],
       [ 1189.   ,     0.   ,     0.164,     2.   ,     2.222],
       [ 1193.   ,     1.   ,     0.166,     2.   ,     0.037],
       [ 1197.   ,     0.   ,     0.904,

In [108]:
rep_zero_afrqs = np.array(small_data['allele/frequency/replicate/0'])

In [110]:
rep_zero_afrqs[qtl]

array([[   13.   ,     0.101,     0.899,     0.101,     0.899],
       [   27.   ,     0.921,     0.079,     0.079,     0.921],
       [  198.   ,     0.89 ,     0.11 ,     0.11 ,     0.89 ],
       [  278.   ,     0.946,     0.054,     0.054,     0.946],
       [  363.   ,     0.347,     0.653,     0.347,     0.653],
       [  436.   ,     0.234,     0.766,     0.234,     0.766],
       [  441.   ,     0.857,     0.143,     0.143,     0.857],
       [  550.   ,     0.503,     0.497,     0.497,     0.503],
       [  604.   ,     0.243,     0.757,     0.243,     0.757],
       [  735.   ,     0.79 ,     0.21 ,     0.21 ,     0.79 ],
       [  853.   ,     0.765,     0.235,     0.235,     0.765],
       [ 1035.   ,     0.119,     0.881,     0.119,     0.881],
       [ 1041.   ,     0.122,     0.878,     0.122,     0.878],
       [ 1189.   ,     0.368,     0.632,     0.368,     0.632],
       [ 1193.   ,     0.785,     0.215,     0.215,     0.785],
       [ 1197.   ,     0.115,     0.885,

In [87]:
qvalues.ix[qvalues['q'] < 0.05]

,q
604,0.000008
735,0.002299
1189,0.000007
1330,0.037586


In [ ]:
def tassel_results_tables(gwas_file_name, q_values_file_name, 
                              minor_allele_frequency_table, 
                              quantitative_allele_table):
    raw_gwas_results = pd.read_csv(gwas_file_name, sep='\t')
    raw_gwas_results.drop(0, axis=0, inplace=True)
    raw_gwas_results.drop('Trait', axis=1, inplace=True)
    raw_gwas_results.index = np.array(list(map(int, raw_gwas_results.Marker)))
    q_values = pd.read_csv(q_values_file_name, sep='\t')
    q_values.index = np.array(list(map(int, raw_gwas_results.Marker)))
    raw_gwas_results = raw_gwas_results.join(q_values)
    
    assert minor_allele_frequency_table.index.dtype == raw_gwas_results.index.dtype, "Indexes of these tables are different"
    
    raw_gwas_results = raw_gwas_results.join(minor_allele_frequency_table.ix[raw_gwas_results.index, :])
    
    assert quantitative_allele_table.index.dtype == raw_gwas_results.index.dtype, "Indexes of these tables are different"
    
    raw_gwas_results = raw_gwas_results.join(quantitative_allele_table.ix[raw_gwas_results.index, :])
    return raw_gwas_results

In [ ]:
pwd

In [ ]:
cd /home/vakanas/tassel-5-standalone/output/

In [ ]:
ls

In [ ]:
mafrqs = pd.read_csv('epsilon_0_maf_table.txt', sep='\t', index_col=0)

In [ ]:
mafrqs

In [ ]:
qtad = pd.read_csv('epsilon_0_quant_allele_table.txt', sep='\t', index_col=0)

In [ ]:
qtad

In [ ]:
super_table = tassel_results_tables('epsilon_0_out_2.txt', 'epsilon_0_qvalues.txt', mafrqs, qtad)

In [ ]:
super_table.ix[super_table.q < 0.05]

In [ ]:
super_table.ix[super_table.alpha_effect > 0]

In [ ]:
mg.multiple_sample_analyzer(meta_populations, qtl, allele_effects, 
                            minor_alleles, concordant_segregating_loci)

In [ ]:
analyze.store_allele_effect_frequency_tables(meta_population, alleles, 
                                             qtl,
                                             exponential_allele_effects,
                                            run_id, 'exponential')

In [ ]:
loci_conversions = shelve.open(run_id+'_loci_conversions')
saegus_to_tassel_loci = {}
tassel_to_saegus_loci = {}
for idx, locus in enumerate(concordant_segregating_loci):
    saegus_to_tassel_loci[locus] = idx
    tassel_to_saegus_loci[idx] = locus
loci_conversions['saegus_to_tassel'] = saegus_to_tassel_loci
loci_conversions['tassel_to_saegus'] = tassel_to_saegus_loci
loci_conversions.close()

In [ ]:
seg_loc_storage = shelve.open('segregating_loci_storage')
seg_loc_storage['bacchus'] = concordant_segregating_loci
seg_loc_storage.close()

In [ ]:
int_to_snp = {0: 'A', 1: 'C', 2: 'G', 3: 'T', 4: '-', 5: '+'}
snp_to_int = {'A': 0, 'C': 1, '-': 4, 'G': 2, '+': 5, 'T': 3}
conv = shelve.open('synthesis_parameters')
conv['integer_to_snp'] = int_to_snp
conv['snp_to_integer'] = snp_to_int
conv.close()

In [ ]:
exponential_allele_effects_table = analyze.generate_allele_effects_table(qtl, alleles, 
                                                exponential_allele_effects, saegus_to_tassel_loci)

In [ ]:
analyze.remap_allele_frequency_table_loci(analyze.reload_allele_frequencies_table(run_id, 0, 250, 
                                                                                  'exponential'), 
                                          concordant_segregating_loci)

In [ ]:
analyze.write_multiple_sample_analyzer(sample_library, sample_sizes, qtl, alleles, 
                                       exponential_allele_effects, 0.7,  concordant_segregating_loci, 
                                       run_id=run_id, sub_run_id='_exponential', 
                                       allele_frequency_hdf=run_id+'_allele_frequency_storage.h5')

In [ ]:
multiple_sample_analyzer(meta_populations, qtl, allele_effects, minor_alleles, concordant_segregating_loci)

In [ ]:
import h5py

In [ ]:
with h5py.File('bia_allele_frequencies.hdf5') as biaf:
    reloaded_af = np.array(biaf[afname])

In [ ]:
fisegloc = list(concordant_segregating_loci)

In [ ]:
minor_allele_frequencies = reloaded_af[fisegloc]

In [ ]:
def write_super_tables(power_and_fpr_raw_data, sample_sizes, number_of_replicates, run_id, sub_run_id=''):
    for size in sample_sizes:
        for rep in range(number_of_replicates):
            name = run_id + '_' + sub_run_id + '_' + str(rep) + '_' + str(size) + '_super_table.txt'
            power_and_fpr_raw_data[size][rep].to_csv(name, sep='\t')

In [ ]:
expo_power_fpr_raw_data = analyze.collect_power_analysis_data(run_id, sample_sizes, number_of_replicates, concordant_segregating_loci, 'exponential')

In [ ]:
expo_power_fpr_raw_data[250]

In [ ]:
write_super_tables(expo_power_fpr_raw_data,
                  sample_sizes,
                  number_of_replicates,
                  'bacchus',
                  sub_run_id='exponential')

In [ ]:
expo_results, expo_true_positives, expo_false_positives = study.calculate_power_fpr(expo_power_fpr_raw_data, sample_sizes, 
                                                                             number_of_replicates, number_of_qtl)

In [ ]:
expo_results

In [ ]:
mean_and_stdev = pd.DataFrame([expo_results.mean(), expo_results.std()], index=['mean', 'stdev']).T
mean_and_stdev.to_csv('bacchus_exponential_mean_and_stdev_power_fpr.csv', sep='\t')

In [ ]:
geo_results

In [ ]:
geometric_allele_effects_table

In [ ]:
exponential_allele_effects_table

In [ ]:
expo_results.to_csv("bacchus_exponential_power_fpr_results.txt", sep='\t')

In [ ]:
mean_and_stdev = pd.DataFrame([geo_results.mean(), geo_results.std()], index=['mean', 'stdev']).T
mean_and_stdev.to_csv('full_icecrown_geometric_mean_and_stdev_power_fpr.txt', sep='\t')

In [ ]:
expo_results, expo_true_positives, expo_false_positives = full_icecrown.calculate_power_fpr(expo_power_fpr_raw_data,
                                                                                      sample_sizes,
                                                                                      number_of_replicates,
                                                                                      number_of_qtl)

In [ ]:
expo_results

In [ ]:
expo_results.to_csv('full_icecrown_exponential_power_fpr_results.txt', sep='\t')

In [ ]:
mean_and_stdev = pd.DataFrame([expo_results.mean(), expo_results.std()], index=['mean', 'stdev']).T
mean_and_stdev.to_csv('full_icecrown_exponential_mean_and_stdev_power_fpr.txt', sep='\t')

In [ ]:
write_super_tables(expo_power_fpr_raw_data, sample_sizes, number_of_replicates, run_id, 'exponential')

In [ ]:
geo_aggregate_estimated_actual = pd.DataFrame([np.array(geo_agg_estimated), np.array(geo_agg_actual)], index=['estimated', 'actual']).T

In [ ]:
geo_aggregate_estimated_actual['estimated'] = geo_aggregate_estimated_actual['estimated'].apply(np.fabs)

In [ ]:
geo_aggregate_estimated_actual

In [ ]:
geo_corr = geo_aggregate_estimated_actual['estimated'].corr(geo_aggregate_estimated_actual['actual'])

In [ ]:
geo_agg_estimated

In [ ]:
aggregate_estimated_actual

In [ ]:
geo_corr

In [ ]:
pwd

In [ ]:
geo_aggregate_estimated_actual.to_csv('full_icecrown_geometric_estimated_vs_actual_allele_effects.txt', sep='\t')

In [ ]:
agg_estimated = []
agg_actual = []

In [ ]:
for rep in reps:
    for size in sample_sizes:
        sutable = sutable_collection[rep][size]
        droppable = list(sutable.ix[sutable.ix[:, 'difference'] == 0.0].index)
        qtloci = sutable.drop(droppable, axis=0)
        agg_estimated.extend(list(qtloci['add_effect']))
        agg_actual.extend(list(qtloci['difference']))

In [ ]:
aggregate_estimated_actual = pd.DataFrame([np.array(agg_estimated), np.array(agg_actual)], index=['estimated', 'actual']).T

In [ ]:
aggregate_estimated_actual['estimated'] = np.fabs(aggregate_estimated_actual['estimated'])

In [ ]:
aggregate_estimated_actual

In [ ]:
correlation_actual_vs_effects = aggregate_estimated_actual['estimated'].corr(aggregate_estimated_actual['actual'])

In [ ]:
aggregate_estimated_actual.to_csv('full_icecrown_exponential_estimated_vs_actual_allele_effects.txt', sep='\t')

In [ ]:
aggregate_estimated_actual['estimated'] = np.fabs(aggregate_estimated_actual['estimated'])

In [ ]:
cd C:\tassel\output\full_icecrown\exponential

In [ ]:
expo_estimated_actual = pd.read_csv('full_icecrown_exponential_estimated_vs_actual_allele_effects.txt', sep='\t', index_col=0)

In [ ]:
expo_estimated_actual

In [ ]:
aggregate_estimated_actual

In [ ]:
, from bokeh.plotting import figure, show, output_file
from bokeh.io import output_notebook

In [ ]:
output_notebook()

In [ ]:
aggregate_estimated_actual

In [ ]:
geo_x = aggregate_estimated_actual['estimated']
geo_y = aggregate_estimated_actual['actual']

In [ ]:
p = figure(title="Estimated vs Actual Allele Effects - Geometric Series", 
           title_text_font_size="16",
          x_range=(-0.2, 4))

In [ ]:
p.scatter(geo_x, y, x="Estimated", y="Actual")

p.xaxis.axis_label = "Estimated"
p.yaxis.axis_label = "Actual"

In [ ]:
show(p)

In [ ]:
expo

In [ ]:
p = figure(title="Estimated vs Actual Allele Effects - Geometric Series", title_text_font_size="16")

In [ ]:
expo_plot = figure(title="Estimated vs Actual Effects - Exponential(lambda=1)", 
                   title_text_font_size="16", 
                  x_range=(0, 4))

x = np.array(expo_estimated_actual['estimated'])
y = np.array(expo_estimated_actual['actual'])

expo_plot.xaxis.axis_label = "Estimated"
expo_plot.yaxis.axis_label = "Actual"

In [ ]:
expo_plot.scatter(x, y)

In [ ]:
show(expo_plot)

In [ ]:
from bokeh.io import hplot

In [ ]:
geo_plot = figure(title="Estimated vs Actual Allele Effects - Geometric Series", 
           title_text_font_size="16",
          x_range=(0, 4), y_range=(0, 4))

In [ ]:
geo_x = aggregate_estimated_actual['actual']
geo_y = aggregate_estimated_actual['estimated']

In [ ]:
geo_plot.xaxis.axis_label = "Actual"
geo_plot.yaxis.axis_label = "Estimated"
geo_plot.scatter(geo_x, geo_y, x="Actual", y="Estimated")

In [ ]:
expo_plot = figure(title="Estimated vs Actual Effects - Exponential(lambda=1)", 
                   title_text_font_size="16", 
                  x_range=(0, 4), y_range=(0, 4))

expo_x = np.array(expo_estimated_actual['actual'])
expo_y = np.array(expo_estimated_actual['estimated'])

expo_plot.xaxis.axis_label = "Actual"
expo_plot.yaxis.axis_label = "Estimated"
expo_plot.scatter(expo_x, expo_y)

In [ ]:
multi_plot = hplot(geo_plot, expo_plot)
show(multi_plot)

In [ ]:
output_file("multi_plot.png")

In [ ]:
ls